In [ ]:
"""
x0=[ 5.0 ,  1.0 ,  0. ,  0.,  0.,  0.,  0.,   0. ,  0. ,  0.  , 0. ,  0.  , 0. ,  0. , 0. ,  0. ,  0. ,  0. ,  0. ,  0. ]
station.GetInputPort("spot.desired_state").FixValue(context, x0)
meshcat.StartRecording()
x0 = station.GetOutputPort("spot.state_estimated").Eval(context)
print(x0)
x0[0]+=1.0
#x0[7]+=0.1
#x0=[ 3. ,  0. ,  0. ,  0.,  0.,  0.,  0.,   0. ,  0. ,  0.  , 0. ,  0.  , 0. ,  0. , 0. ,  0. ,  0. ,  0. ,  0. ,  0. ]
station.GetInputPort("spot.desired_state").FixValue(context, x0)
simulator.AdvanceTo(10.0);

meshcat.PublishRecording()

    - add_model:
        name: goal
        file: file://{goal_model}
        default_free_body_pose:
            Soccergoal:
                translation: [0., -1, 0.]
                rotation: !Rpy {{ deg: [90, 0, 180] }} 
"""

In [ ]:
"""
for camera_name, camera_config in cameras.items():
    # Extract camera parameters
    X_BC = camera_config["X_BC"]
    X_PB = camera_config["X_PB"]
    center_x = camera_config["center_x"]
    center_y = camera_config["center_y"]
    focal = camera_config["focal"]
    width = camera_config["width"]
    height = camera_config["height"]

    # Add RgbdSensor for the camera
    camera_sensor = builder.AddSystem(
        RgbdSensor(
            parent_id=robot.GetBodyByName(camera_name + "_camera_link").index(),
            X_PB=X_PB,
            X_BC=X_BC,
            properties=RgbdSensor.CameraProperties(
                width=width,
                height=height,
                fov_y=np.arctan2(height / 2, focal["y"]) * 2,
                focal=focal,
                k=np.array([focal["x"], focal["y"], center_x, center_y]),
                depth_range=[0.1, 10.0],  # Adjust the depth range as needed
            ),
        )
    )

    # Connect the camera sensor to the robot
    builder.Connect(robot.get_output_port(0), camera_sensor.get_input_port(0)) """

In [ ]:
"""def run_episode(plant, plant_context, TIME, sim_time_step = .1, max_sim = 20 ):
    reset_episode(plant, plant_context)
    meshcat.StartRecording()
    time_ = 0
    current_attacker_index = 0
    #states shooting, moving, passing, collecting_ball, waiting
    state = "moving"
    forces = []
    shot_time = 0
    MAX_TIME_SINCE_SHOT = 5 
    passing_force= None
    x0=[ 5.0 ,  1.0 ,  0. ,  0.,  0.,  0.,  0.,   0. ,  0. ,  0.  , 0. ,  0.  , 0. ,  0. , 0. ,  0. ,  0. ,  0. ,  0. ,  0. ]
    station.GetInputPort("spot.desired_state").FixValue(context, x0)

    
    while True:
        move_left(time_ +TIME)


        if time_ >= 50:
            break
    TIME += time_
    print("Goal", goal)
    meshcat.PublishRecording()
    return TIME"""

In [ ]:
"""def move_hand(plant,plant_context,  desired_target =[1,1,1] , tollerance = 0.05):

    ik = InverseKinematics(plant, plant_context)
    prog = ik.get_mutable_prog()
 
    # Set the target frame for the end effector
    end_effector_frame = plant.GetFrameByName("arm_link_fngr")
    ik.AddPositionConstraint(
        frameA=plant.world_frame(),
        frameB=end_effector_frame,
        p_BQ=np.zeros(3),
        p_AQ_lower=[t -tollerance for t in desired_target ],  # Lower bound for position constraint
        p_AQ_upper=[t +tollerance for t in desired_target ],  # Upper bound for position constraint
    )


    # Set the initial guess for joint positions
    current_q = plant.GetPositions(plant_context)
    q_init_guess = current_q
    q = ik.q()
    prog.AddConstraint(eq(current_q[20:],q[20:]))
    prog.AddConstraint(eq(current_q[:3],q[:3]))
    prog.SetInitialGuess(q, q_init_guess)
    prog.AddQuadraticErrorCost(
        np.eye(len(q)), current_q, q
    )

    #min distance collision constraint, mobile base ik
    ik.AddMinimumDistanceLowerBoundConstraint(0.01)
    #print list of collisions, iterate over collision pairs, measure signed distance
    #scene graph query obj
    #collision filtering 

    result = Solve(ik.prog())
    if result.is_success():
        q_solution = result.GetSolution()
       # station.GetInputPort("spot.desired_state").FixValue(context, q_solution[:20])
        print("moving hand")
        #print("IK solution:", q_solution)
        return True
    else:
        print("failed to move hand jittering")
        return False
        #print("IK failed to converge.")



def move_hand(plant, plant_context, desired_velocity=[0, 0, 0], tolerance=0.05):
    # Set the target frame for the end effector
    end_effector_frame = plant.GetFrameByName("arm_link_fngr")

    diff_ik = DifferentialInverseKinematicsIntegrator(plant,end_effector_frame,time_step=0.1, plant_context)
    prog = diff_ik.get_mutable_prog()

    # Add end-effector velocity constraint
    diff_ik.AddVEqualityToPositionsConstraint(
        frameA=plant.world_frame(),
        frameB=end_effector_frame,
        p_AQ=np.zeros(3),
        p_AQ_lower=[t - tolerance for t in desired_velocity],  # Lower bound for velocity constraint
        p_AQ_upper=[t + tolerance for t in desired_velocity],  # Upper bound for velocity constraint
    )

    # Set the initial guess for joint velocities
    current_v = plant.GetVelocities(plant_context)
    v_init_guess = current_v
    v = diff_ik.v()
    prog.SetInitialGuess(v, v_init_guess)
    prog.AddQuadraticErrorCost(np.eye(len(v)), current_v, v)

    # Minimize joint accelerations (optional)
    prog.AddQuadraticErrorCost(np.eye(len(v)), np.zeros_like(v), v, weight=1.0)

    # Minimize joint torques (optional)
    # prog.AddQuadraticErrorCost(np.eye(len(v)), np.zeros_like(v), v, weight=1.0)

    # Minimize joint positions (optional)
    # prog.AddQuadraticErrorCost(np.eye(len(v)), np.zeros_like(v), v, weight=1.0)

    # Additional constraints, collision avoidance, etc. can be added here

    result = Solve(diff_ik.prog())
    if result.is_success():
        v_solution = result.GetSolution()
        # Integrate joint velocities to get joint positions
        q_solution = plant.CalcQFromV(plant_context, v_solution)
        plant.SetPositions(plant_context, q_solution)
        print("moving hand")
        # print("DiffIK solution:", v_solution)
        return True
    else:
        print("failed to move hand jittering")
        return False
        # print("DiffIK failed to converge.")

"""

In [ ]:
#####IIWA
from pydrake.all import *

def solve_ik(X_WG, max_tries=10, fix_base=False, base_pose=np.zeros(3)):
    diagram, plant, scene_graph = build_env()

    gripper_frame = plant.GetFrameByName("l_gripper_palm_link")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    sg_context = scene_graph.GetMyContextFromRoot(context)
    filterCollsionGeometry(scene_graph, sg_context)

    # Note: passing in a plant_context is necessary for collision-free constraints!
    ik = InverseKinematics(plant, plant_context)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram
    q_nominal = np.zeros(len(q_variables))
    q_nominal[0:3] = base_pose
    prog.AddQuadraticErrorCost(
        np.eye(len(q_variables)), q_nominal, q_variables
    )
    
    # Position constraint
    ik.AddPositionConstraint(
        frameA=plant.world_frame(),
        frameB=gripper_frame,
        p_BQ=np.zeros(3),
        p_AQ_lower=X_WG.translation() - 0.001,  # Lower bound for position constraint
        p_AQ_upper=X_WG.translation() + 0.001,  # Upper bound for position constraint
    )

    # Orientation constraint using orientation error and bounds
    R_WG = X_WG.rotation()
    orientation_tol = np.radians(1.0)  # 1 degree in radians
    ik.AddOrientationConstraint(
        frameAbar=plant.world_frame(),
        R_AbarA=R_WG,
        frameBbar=gripper_frame,
        R_BbarB=RotationMatrix(),
        theta_bound=orientation_tol,
    )

    if fix_base:
        # Constrain the base pose (x, y, theta) to be equal to base_pose
        prog.AddConstraint(eq(base_pose,q_variables[:3]))

    ik.AddMinimumDistanceLowerBoundConstraint(0.01)

    for count in range(max_tries):
        # Compute a random initial guess for joint angles within joint limits
        q_initial_guess = np.zeros(len(q_variables))
        for i in range(len(q_variables)):
            joint = np.zeros(len(q_variables))
            lowerlim = plant.GetPositionLowerLimits()
            upperlim = plant.GetPositionUpperLimits()
            for i in range(len(q_variables)):
                if lowerlim[i] is None or upperlim[i]==np.inf:
                    joint[i] = np.random.uniform(-np.pi, np.pi)
                else:
                    joint[i] = np.random.uniform(lowerlim[i], upperlim[i])

        prog.SetInitialGuess(q_variables, joint)
        result = Solve(prog)

        if result is not None and result.is_success():
            print("Succeeded in %d tries!" % (count + 1))
            return result.GetSolution(q_variables)

    print("Failed!")
    return None


In [ ]:
#    v_now = plant.GetVelocities(plant_context)
#    p_now = plant.GetPositions(plant_context)
#    end_effector_frame = plant.GetFrameByName("arm_link_fngr")
#    world_frame = plant.world_frame()
#    G = plant.GetBodyByName("body").body_frame()
#    J_G = plant.CalcJacobianSpatialVelocity(plant_context,
#            JacobianWrtVariable.kQDot,G,
#            [0, 0, 0],W,W,)

#    q = DiffIKQP(J_G,np.repeat(1, 6), v_now , p_now)



In [ ]:
def DiffIKQP(J_G, V_G_desired, v_now, p_now, target =[1,1,1], tolerace= .5):
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(10, "v")
    v_max = 3.0  # do not modify
    h = 4e-3  # do not modify

    # Add cost and constraints to prog here.
    z=(np.linalg.norm(J_G.dot(v)-V_G_desired))**2
    prog.AddCost(z)



    # Joint velocity constraints
    prog.AddConstraint(le(v, np.repeat(v_max, 10)))
    prog.AddConstraint(ge(v, np.repeat(-1*v_max, 10)))

    # End-effector position constraints
    lower_bound = np.array([t - tolerace for t in target ])  
    upper_bound = np.array([t + tolerace for t in target ]) 
    
    p_estimated = p_now + v * .1
    hand_estimate 
    prog.AddConstraint(le(p, upper_bound))
    prog.AddConstraint(ge(p, lower_bound))

    solver = SnoptSolver()
    result = solver.Solve(prog)
    if not (result.is_success()):
        raise ValueError("Could not find the optimal solution.")

    v_solution = result.GetSolution(v)

    command_v = v_solution
    command_p =p_now + v_solution*.1
    x0 = np.concatenate([command_p, command_v])
    station.GetInputPort("spot.desired_state").FixValue(context, x0)

    #end_effector_frame = plant.GetFrameByName("arm_link_fngr")
#params = DifferentialInverseKinematicsParameters(10,10)
##diffIK = DifferentialInverseKinematicsIntegrator(plant, end_effector_frame, .1, params)
#builder.AddSystem(diffIK)

    return v_solution

In [ ]:
def move_hand_old(plant, plant_context):
        v_now = plant.GetVelocities(plant_context)[10:20]
        p_now = plant.GetPositions(plant_context)[:10]
        V_G_desired = np.ones(6)
        J_G = plant.CalcJacobianSpatialVelocity(
            plant_context,
            JacobianWrtVariable.kQDot,
            plant.GetBodyByName("body").body_frame(),
            [0, 0, 0],
            plant.world_frame(),
            plant.world_frame(),
        )
        J_G = J_G[:, :10]

        DiffIKQP(J_G, V_G_desired, v_now, p_now)

In [ ]:
def move_right(x0, pos= 1, vel=.1):
    desired_state = x0
    desired_state[8] -= vel  # Adjust the linear velocity as needed in the x-direction
    desired_state[1] -= pos  # Optionally, adjust the x-position to move forward
    return desired_state

In [ ]:
def move_left(x0, pos= 1, vel=.1):
    desired_state = x0
    desired_state[8] = vel  # Adjust the linear velocity as needed in the x-direction
    desired_state[1] += pos  # Optionally, adjust the x-position to move forward
    return desired_state

In [ ]:
def move_backward(x0, pos= 1, vel=.1):
    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[7] -= vel  # Adjust the linear velocity as needed in the x-direction
    desired_state[0] -= pos  # Optionally, adjust the x-position to move forward
    return desired_state

In [ ]:
def move_forward(x0, pos= 1, vel=.1):
    # Modify the desired state to command forward movement
    desired_state = x0
    desired_state[7]+= vel  # Adjust the linear velocity as needed in the x-direction
    desired_state[0] += pos # Optionally, adjust the x-position to move forward
    return desired_state

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=443541ce-e170-4459-8340-438ed0837988' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>